In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data=datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
batch_size=64
train_dataloader=DataLoader(training_data,batch_size=batch_size)
test_dataloader=DataLoader(test_data,batch_size=batch_size)

for X,y in test_dataloader:
    print(f"Shape of X [N,C,H,W]:{X.shape}")
    print(f"Shape of y:{y.shape} {y.dtype}")
    break

Shape of X [N,C,H,W]:torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]) torch.int64


In [4]:
device=(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [5]:

print(device)

cpu


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )

    def forward(self,x):
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits

model=NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=1e-3)

In [8]:
def train(dataloader,model,loss_fn,optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch,(X,y) in enumerate(dataloader):
        X,y=X.to(device),y.to(device)
        pred=model(X)
        loss=loss_fn(pred,y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 100==0:
            loss,current=loss.item(),(batch+1)*len(X)
            print(f"loss:{loss:>7f}[{current:>5d}|{size:>5d}]")

In [9]:
def test(dataloader,model,loss_fn):
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    model.eval()
    test_loss,correct=0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y=X.to(device),y.to(device)
            pred=model(X)
            test_loss+=loss_fn(pred,y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
        test_loss/=num_batches
        correct/=size
        print(f"Test Error:\n Accuracy:{100*correct:>0.1f}%, Avg loss:{test_loss:>8f}\n")

In [10]:
epoches=10
for t in range(epoches):
    print(f"Epoch {t+1}\n-------------------")
    train(train_dataloader,model,loss_fn,optimizer)
    test(test_dataloader,model,loss_fn)
print("Done!")

Epoch 1
-------------------
loss:2.306598[   64|60000]
loss:2.292994[ 6464|60000]
loss:2.264856[12864|60000]
loss:2.258965[19264|60000]
loss:2.244681[25664|60000]
loss:2.218702[32064|60000]
loss:2.219215[38464|60000]
loss:2.185597[44864|60000]
loss:2.178462[51264|60000]
loss:2.139923[57664|60000]
Test Error:
 Accuracy:47.9%, Avg loss:2.141292

Epoch 2
-------------------
loss:2.151059[   64|60000]
loss:2.142791[ 6464|60000]
loss:2.069249[12864|60000]
loss:2.089998[19264|60000]
loss:2.038016[25664|60000]
loss:1.975318[32064|60000]
loss:2.000074[38464|60000]
loss:1.912492[44864|60000]
loss:1.921550[51264|60000]
loss:1.840335[57664|60000]
Test Error:
 Accuracy:57.5%, Avg loss:1.843347

Epoch 3
-------------------
loss:1.875695[   64|60000]
loss:1.850901[ 6464|60000]
loss:1.714325[12864|60000]
loss:1.764204[19264|60000]
loss:1.658986[25664|60000]
loss:1.604299[32064|60000]
loss:1.625854[38464|60000]
loss:1.523128[44864|60000]
loss:1.558316[51264|60000]
loss:1.451685[57664|60000]
Test Error

In [11]:
torch.save(model.state_dict(),"model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [12]:
import torch
import torchvision.models as models
model=NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [13]:
classes=[
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

model.eval()
x,y=test_data[1][0],test_data[1][1]
with torch.no_grad():
    x=x.to(device)
    pred=model(x)
    predicted,actual=classes[pred[0].argmax(0)],classes[y]
    print(f'Precited:"{predicted}",Actual:"{actual}"')


Precited:"Pullover",Actual:"Pullover"


In [1]:
import torch
import torchvision.models as models
model=models.vgg16(weights="IMAGENET1K_V1")
torch.save(model.state_dict(),'model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\ruilong.zhao/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100.0%


In [2]:
model=models.vgg16()
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1